# Pre-work TikTok soundcharts API

## Setup

In [1]:
# Import modules
import pandas as pd

from datetime import datetime, timedelta
from soundcharts_API_client import *
from tqdm import tqdm


In [2]:
# API connexion
x_app_id = 'PARIS1_E22B0B40'
x_api_key = '3c7bfb46ee2705df'

sdch = SoundchartsClient(x_app_id, x_api_key)


## Get data

### Get track and artist-level infos

In [ ]:
# Import playlist data
df = pd.read_excel('tiktok_charts_tracks_with_use.xlsx')


In [ ]:
# Get the tracks' and artists' info
results_df = []

for index, row in df.iterrows():
    track_id = row['track_uuid']
    tiktok_id = row['track_tiktok_id']
    
    collection_date = row['collection_date']
    track_position = row['track_position']
    artist_video = row['artist_video']
    artist_video_date = row['artist_video_date']

    tracks = sdch.get_track_metadata(track_id=track_id)

    track_names = tracks.get("object", {}).get("name")
    tracks_ids = tracks.get("object", {}).get("uuid")
    track_dates = tracks.get("object", {}).get("releaseDate")
    track_features = tracks.get("object", {}).get("audio")

    artist_names = []
    artist_ids = []

    for artist in tracks.get("object", {}).get("artists", []):
        artist_names.append(artist.get("name"))
        artist_ids.append(artist.get("uuid"))

    label_names = []
    label_types = []

    for label in tracks.get("object", {}).get("labels", []):
        label_names.append(label.get("name"))
        label_types.append(label.get("type"))
    
    track_genres = []
    
    for genre in tracks.get("object", {}).get("genres", []):
        track_genres.append(genre.get("root"))

    collection_date_dt = datetime.fromisoformat(collection_date)
    date_norm = collection_date_dt.strftime('%Y-%m-%d')

    followers = []

    for artist_id in artist_ids:
        artists_followers = sdch.get_artist_tiktok_followers_by_date(
            artist_id=artist_id,
            startDate=(collection_date_dt - timedelta(days=1)).strftime('%Y-%m-%d'),
            endDate=date_norm
        )

        if 'items' in artists_followers and artists_followers['items']:
            follower_count = artists_followers['items'][0].get('followerCount', None)
            followers.append(follower_count)
        else:
            followers.append(None)

    results_df.append({
        "collection_date": collection_date,
        "track_position": track_position,
        "track_name": track_names,
        "track_id": tracks_ids,
        "track_tiktok_id": tiktok_id,
        "artist_name": artist_names,
        "artist_id": artist_ids,
        "artist_followers": followers,
        "label_name": label_names,
        "label_type": label_types,
        "track_date": track_dates,
        "track_genres": track_genres,
        "track_features": track_features,
        "artist_video": artist_video,
        "artist_video_date": artist_video_date
    })

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_csv('tiktok_tracks_info.csv', index=False)

# Print the DataFrame
display(df)


### Get tracks' streams and tiktok videos

In [5]:
# Import tracks playlist data
df = pd.read_csv('tiktok_tracks_info.csv')


In [6]:
# Initialize an empty list to store results
results_df = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    collection_date = row['collection_date']
    track_id = row['track_id']
    tiktok_id = row['track_tiktok_id']
    track_name = row['track_name']
    track_position = row['track_position']
    track_date = row['track_date']
    track_genres = row['track_genres']
    track_features = row['track_features']
    artist_name = row['artist_name']
    artist_id = row['artist_id']
    artist_followers = row['artist_followers']
    label_name = row['label_name']
    label_type = row['label_type']
    artist_video = row['artist_video']
    artist_video_date = row['artist_video_date']

    if pd.isna(artist_video_date) or artist_video_date is None:
        artist_video_date = collection_date 

    try:
        artist_video_date_dt = datetime.fromisoformat(artist_video_date)
        start_date = (artist_video_date_dt - timedelta(days=45)).strftime('%Y-%m-%d')
        end_date = (artist_video_date_dt + timedelta(days=44)).strftime('%Y-%m-%d')

        # Fetch Spotify streams
        streams_data = sdch.get_track_streams_by_date(
            startDate=start_date,
            endDate=end_date,
            track_id=track_id
        )

        # Fetch TikTok video streams
        tiktok_streams_data = sdch.get_tiktok_streams_by_date(
            endDate=end_date,
            period=90,  # Fetching 90 days of TikTok data
            track_id=tiktok_id
        )

        processed_date = set()

        for item in streams_data.get("items", []):
            date = item.get("date")
            if date in processed_date:
                continue
            
            processed_date.add(date)
            
            first_plot = next(iter(item.get("plots", [])), None)
            spotify_streams = first_plot.get("value") if first_plot else None

            # Extract TikTok streams for the same date
            tiktok_streams_value = None
            
            if "items" in tiktok_streams_data and tiktok_streams_data["items"]:
                for tiktok_item in tiktok_streams_data["items"]:
                    tiktok_date = tiktok_item.get("date")
                    if tiktok_date == date:
                        tiktok_streams_value = tiktok_item.get("value", None)
                        tiktok_stream_date = tiktok_date
                        break 
                    
            results_df.append({
                "collection_date": collection_date,
                "track_id": track_id,
                "track_tiktok_id": tiktok_id,
                "track_name": track_name,
                "track_position": track_position, 
                "track_date": track_date, 
                "track_genres": track_genres, 
                "track_features": track_features, 
                "artist_name": artist_name,
                "artist_id": artist_id, 
                "artist_followers": artist_followers, 
                "label_name": label_name, 
                "label_type": label_type, 
                "artist_video": artist_video, 
                "artist_video_date": artist_video_date, 
                "stream_date": date,
                "spotify_streams": spotify_streams,
                "tiktok_stream_date": tiktok_date,
                "tiktok_streams": tiktok_streams_value,
            })

    except Exception as e:
        print(f"Error for track {track_id}: {e}")

# Convert the list to a DataFrame
df_results = pd.DataFrame(results_df)
df_results.to_csv('tiktok_streams.csv', index=False)

# Print the DataFrame
display(df_results)


Processing rows:   0%|          | 1/300 [00:00<01:52,  2.66it/s]

Status code: 200
Status code: 200


Processing rows:   1%|          | 2/300 [00:00<01:42,  2.90it/s]

Status code: 200
Status code: 200


Processing rows:   1%|          | 3/300 [00:01<01:39,  3.00it/s]

...

Processing rows: 100%|█████████▉| 299/300 [01:37<00:00,  3.38it/s]

Status code: 200
Status code: 200


Processing rows: 100%|██████████| 300/300 [01:37<00:00,  3.08it/s]

Status code: 200
Status code: 200


,collection_date,track_id,track_tiktok_id,track_name,track_position,track_date,track_genres,track_features,artist_name,artist_id,artist_followers,label_name,label_type,artist_video,artist_video_date,stream_date,spotify_streams,tiktok_stream_date,tiktok_streams
0,2024-09-01T00:00:00+02:00,5a4ae67e-d91c-4883-95cb-20035f2a3f7a,7403552691299633168,Taste,1,2024-08-23T00:00:00+00:00,['pop'],NaN,['Sabrina Carpenter'],['11e81bba-57e6-3220-a3c7-a0369fe50396'],[31500000],['Island'],['Universal'],True,2024-08-25,2024-10-08T00:00:00+00:00,318651503,2024-08-24T00:00:00+00:00,NaN
1,2024-09-01T00:00:00+02:00,5a4ae67e-d91c-4883-95cb-20035f2a3f7a,7403552691299633168,Taste,1,2024-08-23T00:00:00+00:00,['pop'],NaN,['Sabrina Carpenter'],['11e81bba-57e6-3220-a3c7-a0369fe50396'],[31500000],['Island'],['Universal'],True,2024-08-25,2024-10-07T00:00:00+00:00,313216563,2024-08-24T00:00:00+00:00,NaN
2,2024-09-01T00:00:00+02:00,5a4ae67e-d91c-4883-95cb-20035f2a3f7a,7403552691299633168,Taste,1,2024-08-23T00:00:00+00:00,['pop'],NaN,['Sabrina Carpenter'],['11e81bba-57e6-3220-a3c7-a0369fe50396'],[31500000],['Island'],['Universal'],True,2024-08-25,2024-10-06T00:00:00+00:00,307119726,2024-10-06T00:00:00+00:00,2600000.0
3,2024-09-01T00:00:00+02:00,5a4ae67e-d91c-4883-95cb-20035f2a3f7a,7403552691299633168,Taste,1,2024-08-23T00:00:00+00:00,['pop'],NaN,['Sabrina Carpenter'],['11e81bba-57e6-3220-a3c7-a0369fe50396'],[31500000],['Island'],['Universal'],True,2024-08-25,2024-10-05T00:00:00+00:00,300681297,2024-08-24T00:00:00+00:00,NaN
4,2024-09-01T00:00:00+02:00,5a4ae67e-d91c-4883-95cb-20035f2a3f7a,7403552691299633168,Taste,1,2024-08-23T00:00:00+00:00,['pop'],NaN,['Sabrina Carpenter'],['11e81bba-57e6-3220-a3c7-a0369fe50396'],[31500000],['Island'],['Universal'],True,2024-08-25,2024-10-04T00:00:00+00:00,294398878,2024-10-04T00:00:00+00:00,2600000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19792,2024-09-01T00:00:00+02:00,1e23be4e-7232-4f8d-b342-e771ed94c2a9,7093206384738437893,Damn,459,2020-05-22T00:00:00+00:00,"['pop', 'r&b, funk & soul']","{'acousticness': 0.58, 'danceability': 0.72, '...",['Omah lay'],['28810016-6284-11e9-bd7c-549f35141000'],[2500000],"['Sire', 'The KeyQaad']","['Warner', 'unknown']",False,2024-09-01T00:00:00+02:00,2024-07-22T00:00:00+00:00,25997902,2024-07-22T00:00:00+00:00,15200.0
19793,2024-09-01T00:00:00+02:00,1e23be4e-7232-4f8d-b342-e771ed94c2a9,7093206384738437893,Damn,459,2020-05-22T00:00:00+00:00,"['pop', 'r&b, funk & soul']","{'acousticness': 0.58, 'danceability': 0.72, '...",['Omah lay'],['28810016-6284-11e9-bd7c-549f35141000'],[2500000],"['Sire', 'The KeyQaad']","['Warner', 'unknown']",False,2024-09-01T00:00:00+02:00,2024-07-21T00:00:00+00:00,25980744,2024-07-21T00:00:00+00:00,15100.0
19794,2024-09-01T00:00:00+02:00,1e23be4e-7232-4f8d-b342-e771ed94c2a9,7093206384738437893,Damn,459,2020-05-22T00:00:00+00:00,"['pop', 'r&b, funk & soul']","{'acousticness': 0.58, 'danceability': 0.72, '...",['Omah lay'],['28810016-6284-11e9-bd7c-549f35141000'],[2500000],"['Sire', 'The KeyQaad']","['Warner', 'unknown']",False,2024-09-01T00:00:00+02:00,2024-07-20T00:00:00+00:00,25962471,2024-07-20T00:00:00+00:00,15200.0
19795,2024-09-01T00:00:00+02:00,1e23be4e-7232-4f8d-b342-e771ed94c2a9,7093206384738437893,Damn,459,2020-05-22T00:00:00+00:00,"['pop', 'r&b, funk & soul']","{'acousticness': 0.58, 'danceability': 0.72, '...",['Omah lay'],['28810016-6284-11e9-bd7c-549f35141000'],[2500000],"['Sire', 'The KeyQaad']","['Warner', 'unknown']",False,2024-09-01T00:00:00+02:00,2024-07-19T00:00:00+00:00,25945194,2024-07-19T00:00:00+00:00,15200.0
